In [1]:
!pip install spacy==3.0.0
!pip install ml-datasets
!python -m spacy download en_core_web_md

     ---------------------------------------- 11.4/11.4 MB 7.3 MB/s eta 0:00:00
     ---------------------------------------- 96.8/96.8 KB 5.4 MB/s eta 0:00:00
     ---------------------------------------- 1.0/1.0 MB 8.2 MB/s eta 0:00:00
     ---------------------------------------- 42.8/42.8 KB 2.2 MB/s eta 0:00:00
     -------------------------------------- 481.4/481.4 KB 7.5 MB/s eta 0:00:00
     ---------------------------------------- 1.8/1.8 MB 7.5 MB/s eta 0:00:00
     ---------------------------------------- 7.0/7.0 MB 7.9 MB/s eta 0:00:00
     ---------------------------------------- 82.8/82.8 KB ? eta 0:00:00
  Using cached smart_open-5.2.1-py3-none-any.whl (58 kB)
  Attempting uninstall: click
    Found existing installation: click 8.0.3
    Uninstalling click-8.0.3:
      Successfully uninstalled click-8.0.3


You should consider upgrading via the 'C:\Games\conda\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'C:\Games\conda\python.exe -m pip install --upgrade pip' command.


2022-10-22 13:12:19.394402: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'cudart64_110.dll'; dlerror: cudart64_110.dll not found
2022-10-22 13:12:19.395234: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.



     ---------------------------------------- 47.1/47.1 MB 6.2 MB/s eta 0:00:00
[+] Download and installation successful
You can now load the package via spacy.load('en_core_web_md')


You should consider upgrading via the 'C:\Games\conda\python.exe -m pip install --upgrade pip' command.


In [2]:
import spacy
from tqdm.auto import tqdm
from spacy.tokens import DocBin
from ml_datasets import imdb
train_data, valid_data = imdb()
nlp = spacy.load("en_core_web_md")

84131840it [00:50, 1667166.02it/s]                                                                                     


Untaring file...


In [3]:
train_data[0]

("This movie is so bad it's good -- in an unintentionally funny way. I couldn't stop watching it, I was laughing so much! It's like a parody of a romantic thriller, except it's not a parody. \n\n\n\nAlexandra Paul plays Emily Wendell, an oppressed preacher's wife who falls hard for Luke (Corey Sevier), a hunky and mysterious drifter who we eventually learn was in prison; the only thing Sevier is guilty of, though, is bad acting! Mind you, he's no worse than the other actors. You get the sense that the actors have *no* idea they're in a really awful film; they're playing it straight. Everything about the film is bad: the acting, the script, the love scenes, the pacing, the plot twists, the choice of music. The climactic scenes are just so ludicrous -- first the shootout in the church, then Luke's final words to Emily -- I was howling with laughter. \n\n\n\nEvidently Luke did a lot of weight lifting and ab crunches in prison, and we get to see plenty of his naked torso. That's probably t

In [4]:
nlp.pipe_names

['tok2vec', 'tagger', 'parser', 'ner', 'attribute_ruler', 'lemmatizer']

In [5]:
def make_docs(data):
    """
    this will take a list of texts and labels
    and transform them in spacy documents
    data: list(tuple(text, label))
    returns: List(spacy.Doc.doc)
    """
    docs = []
    # nlp.pipe([texts]) is way faster than running
    # nlp(text) for each text
    # as_tuples allows us to pass in a tuple,
    # the first one is treated as text
    # the second one will get returned as it is.
    a = tqdm(nlp.pipe(data, as_tuples=True), total = len(data))
    for doc, label in tqdm(nlp.pipe(data, as_tuples=True), total = len(data)):
        if label == 'neg':
            doc.cats["positive"] = 0
            doc.cats["negative"] = 1
        else:
            doc.cats["positive"] = 1
            doc.cats["negative"] = 0
        docs.append(doc)
    return docs

In [6]:
# это для примера, можно взять больше количество текстов
num_texts = 100
# first we need to transform all the training data
train_docs = make_docs(train_data[:num_texts])

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

In [7]:
# then we save it in a binary file to disc
doc_bin = DocBin(docs=train_docs)
doc_bin.to_disk("train.spacy")
# repeat for validation data
valid_docs = make_docs(valid_data[:num_texts])
doc_bin = DocBin(docs=valid_docs)
doc_bin.to_disk("valid.spacy")

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

на этом месте мы идем в https://spacy.io/usage/training#quickstart, там настраиваем под себя конфиг в разделе quickstart, копируем его руками(!) в base_config.cfg, указываем правильные пути до трейн и вэлид
но после этого некоторые поля возможно придется руками дописать, главное проверить что есть вот такие строчки:

[nlp] <br>
lang = "en" <br>
pipeline = ["textcat"] <br>
batch_size = 1000 <br>

[components] <br>

[components.textcat] <br>
factory = "textcat" <br>

или можно просто взять мой приложенный, он рабочий

In [8]:
# что здесь происходит: мы заполнили base_config, а эта команда на его основе прописывает config.cfg
! python -m spacy init fill-config base_config.cfg config.cfg

2022-10-22 13:22:17.945362: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'cudart64_110.dll'; dlerror: cudart64_110.dll not found
2022-10-22 13:22:17.945403: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
Usage: python -m spacy init fill-config [OPTIONS] BASE_PATH [OUTPUT_FILE]
Try 'python -m spacy init fill-config --help' for help.

Error: Invalid value for 'BASE_PATH': File 'base_config.cfg' does not exist.


In [9]:
! python -m spacy train config.cfg --output ./output # тренькаем модель

2022-10-22 13:22:27.967721: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'cudart64_110.dll'; dlerror: cudart64_110.dll not found
2022-10-22 13:22:27.967761: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
Usage: python -m spacy train [OPTIONS] CONFIG_PATH
Try 'python -m spacy train --help' for help.

Error: Invalid value for 'CONFIG_PATH': Path 'config.cfg' does not exist.


In [10]:
import spacy 
# load thebest model from training
nlp = spacy.load("output/model-best")
text = ""
print("type : ‘quit’ to exit")
# predict the sentiment until someone writes quit
while text != "quit":
    text = input("Please enter example input: ")
    doc = nlp(text)
    print(doc.cats)
    if doc.cats['positive'] >.5:
        print(f"the sentiment is positive")
    else:
        print(f"the sentiment is negative")

OSError: [E050] Can't find model 'output/model-best'. It doesn't seem to be a Python package or a valid path to a data directory.